In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

In [ ]:
SOS_token = 0 # 句子的开始 start of sentence
EOS_token = 1 # 句子的结束 end of sentence

In [ ]:
# 创建一个字典库, 创建数字与单词的一一对应的字典
class LangEmbed:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
# turn a Unicode string to plain ASCII
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# 去除标点, 大写字母换为小写字母
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [ ]:
# 限制数据集的句子长度不大于 10
MAX_LENGTH = 10
eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPairs(pairs):
    p = []
    for pair in pairs:
        if len(pair[0].split(' ')) < MAX_LENGTH and len(pair[1].split(' ')) < MAX_LENGTH and pair[0].startswith(eng_prefixes):
            p.append(pair)
    return p

In [ ]:
def prepareData():
    # Read the file and split into lines
    lines = open('data/eng-fra.txt', encoding='utf-8').read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    fra_lang = LangEmbed("fra")
    eng_lang = LangEmbed("eng")

    print("Read %s sentence pairs" % len(pairs))
    
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    
    for pair in pairs:
        fra_lang.addSentence(pair[1])
        eng_lang.addSentence(pair[0])
    print("Number of Words:")
    print("eng:", eng_lang.n_words)
    print("fra:", fra_lang.n_words)
    return fra_lang, eng_lang, pairs

In [ ]:
# 加载了 135842 条句子, 通过筛选得到 10853 条句子, 其中英文单词总数为 2925, 法文单词总数为 4489
input_lang, output_lang, pairs = prepareData() 

In [ ]:
# 打印两个字典的前 10 个序号所对应的单词
print("English LangEmbed:")
for i in range(10):    
    print(i,":",output_lang.index2word[i])
print("...")

print("French LangEmbed:")
for i in range(10):    
    print(i,":",input_lang.index2word[i])
print("...")

# 在 pairs 中随机选择一组句子进行打印
print(random.choice(pairs)) 

In [ ]:
use_cuda = torch.cuda.is_available()

In [ ]:
# 词嵌入 word embedding - 将单词或者单词序号转换成一个高维向量, 这个高维向量能反映单词的特征, 保持相似性
# nn.Embedding(input_size = 单词的总数, hidden_size = 输出的高维向量的维数)
# 采用 GRU 循环神经网络 - 改善梯度消失
# nn.GRUCell(输入的维数即经过词嵌入的高维向量, 隐藏节点的维数)
# 由于每个句子的长度不一, 选择每次仅输入一个单词, 选择 GRUCell, 输入格式为 (batch, feature) = (1, hidden_size)
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRUCell(hidden_size, hidden_size) # hidden_size 即 embedding 之后的每个词的向量维数, 即中间隐藏节点个数 - 256 维

    def forward(self, input, hidden): # 输入一个单词, 输出结果及隐藏向量, 但其中并没有采用一个序列进行输入
        output = self.embedding(input)
        hidden = self.gru(output, hidden)
        return hidden

    def initHidden(self):
        result = torch.zeros(1, self.hidden_size) # 隐藏向量是（layer*direction, batch, hidden_size）格式, 要写成（1, 256）
        if use_cuda:
            return result.cuda()
        else:
            return result

In [ ]:
# 需要将每次输出的隐藏向量转换成总次数的概率分布向量
# nn.Embedding(output_size = 输出的词汇总数量, hidden_size = 嵌入的词向量的维数)
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRUCell(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)


    def forward(self, input, hidden):
        output = self.embedding(input) #（1, 256）的数据作为输入
        output = F.relu(output)
        hidden = self.gru(output, hidden)
        output = F.log_softmax(self.out(hidden))
        return output, hidden

    def initHidden(self):
        result = torch.zeros(1, self.hidden_size)
        if use_cuda:
            return result.cuda()
        else:
            return result

In [ ]:
def sentence2index(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def indexesFromSentence(lang, sentence):
    indexes = sentence2index(lang, sentence)
    indexes.append(EOS_token)
    result = torch.LongTensor(indexes).view(-1, 1)
    if use_cuda:
        return result.cuda()
    else:
        return result

def indexesFromPair(pair):
    inputs= indexesFromSentence(input_lang, pair[1])
    targets = indexesFromSentence(output_lang, pair[0])
    return (inputs, targets)

In [ ]:
def trainIters(encoder, decoder, n_iters, print_every = 500, learning_rate = 0.01):

    print_loss_total = 0

    encoder_optimizer = optim.SGD(encoder.parameters(), lr = learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr = learning_rate)
    
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        random.shuffle(pairs)
        training_pairs = [indexesFromPair(pair) for pair in pairs]
        
        for idx, training_pair in enumerate(training_pairs):
            input_index = training_pair[0]
            target_index = training_pair[1]

            loss = train(input_index, target_index, encoder,
                         decoder, encoder_optimizer, decoder_optimizer, criterion)
        
            print_loss_total += loss
    
            if idx % print_every == 0:

                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print('iteration:%s, idx:%d, average loss:%.4f' % (iter, idx, print_loss_avg))

In [ ]:
# 在该训练函数中, 先 encoder.initHidden() 初始化编码器的隐藏向量, 再使用 for 循环将输入序列进行编码
# 编码时, 依次将输入序列的元素传入 encoder() 函数, 编码器最终输出隐藏向量 encoder_hidden
# 解码时, 第一个输入元素是 SOS 标识符 0, 将 encoder_hidden 作为解码器的初始隐藏向量
# 解码分两种情况
# 1. 直接将上一次解码输出的词作为下次解码的输入
# 2. 每次均由目标结果作为输入
# 随机选择上述两种方式
teacher_forcing_ratio = 0.5

def train(inputs, targets, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = inputs.size()[0] 
    target_length = targets.size()[0]

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size) # max_length = 10 即不超过十个单词
    encoder_outputs = encoder_outputs.cuda() if use_cuda else encoder_outputs 

    loss = 0
    
    for ei in range(input_length):
        encoder_hidden = encoder(inputs[ei], encoder_hidden)
    
    decoder_input = torch.LongTensor([SOS_token])
    decoder_input = decoder_input.cuda() if use_cuda else decoder_input
    decoder_hidden = encoder_hidden
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, targets[di])
            decoder_input = targets[di]  # Teacher forcing

    else:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1) # 返回最大的数据, 序号
            ni = topi[0][0]

            decoder_input = torch.LongTensor([ni])
            decoder_input = decoder_input.cuda() if use_cuda else decoder_input

            loss += criterion(decoder_output, targets[di])
            if ni == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length # 平均每一个词的 loss

In [ ]:
hidden_size = 256
encoder = EncoderRNN(input_lang.n_words, hidden_size)
decoder = DecoderRNN(hidden_size, output_lang.n_words)

In [ ]:
if use_cuda:
    encoder = encoder.cuda()
    decoder = decoder.cuda()

In [ ]:
trainIters(encoder, decoder, 10)